In [41]:
import torch
import pickle

In [68]:

from modules import obs_encoder, pi_encoder, r_aggregator, r_to_z, z_to_L0, conditional_ODE_func, decoder
from models import LUPI_NDP, NDP

In [6]:
from generators import OscillatorsGenerator, LotkaVolterraGenerator

In [5]:
evaluation_config = {
    # data stuff
    'eval_examples' : 500,
    'simulator' : {
        'dt' : 0.01,
        'sampling_freq' : 20,
        'samples' : 50
    },
    'fixed' : {
        'm1' : 1,
        'm2' : 1,
        'x1' : 1,
        'x2' : -1,
        'v1' : 0,
        'v2' : 0,
        'k' : 0.5,
    },
    'varying' : {
        'c' : [0.5, 2.]
    },
    'eval_varying' : {
        'c' : [0.5, 2.]
    }
}

lv_evaluation_config = {
    # data stuff
    'eval_examples' : 500,
    'simulator' : {
        'dt' : 0.01,
        'sampling_freq' : 20,
        'samples' : 50
    },
    'fixed' : {
        'a' : 2/3,
        'b' : 4/3,
        'c' : 1,
        'd' : 1,
    },
    'varying' : {
        'u1' : [0.2, 1.],
        'v1' : [0.1, 0.5]
    },
    'eval_varying' : {
        'u1' : [0.2, 1.],
        'v1' : [0.1, 0.5]
    },
}

In [4]:
eval_set = OscillatorsGenerator(evaluation_config, train=False)

Evaluation data generation...
Fixed paramters {'m1': 1, 'm2': 1, 'x1': 1, 'x2': -1, 'v1': 0, 'v2': 0, 'k': 0.5}
Varying parameters {'c': [0.5, 2.0]}


In [7]:
lv_eval_set = LotkaVolterraGenerator(lv_evaluation_config, train=False)

Evaluation data generation...
Fixed paramters {'a': 0.6666666666666666, 'b': 1.3333333333333333, 'c': 1, 'd': 1}
Varying parameters {'u1': [0.2, 1.0], 'v1': [0.1, 0.5]}


In [54]:
import yaml

In [71]:
config = yaml.safe_load(open("configs/default.yml"))

In [82]:
models = [1,2]
lists = [[] for _ in models]

In [86]:
lists[0].append(5)

In [87]:
lists

[[5, 5], []]

In [88]:
b = lists

In [89]:
b[1].append(10)

In [93]:
[[]]*5

[[], [], [], [], []]

In [90]:
lists

[[5, 5], [10]]

In [72]:
print(f'Config: {config}')

Config: {'models': 'both', 'use_saved_models': True, 'saved_models': ['lupi_varying_drag', 'nopi_varying_drag'], 'use_saved_data': True, 'saved_train': 'varying_drag_train', 'saved_val': 'varying_drag_validation', 'saved_eval': 'varying_drag_evaluation'}


In [73]:
config['saved_models']

['lupi_varying_drag', 'nopi_varying_drag']

In [74]:
from modules import obs_encoder, pi_encoder, r_aggregator, r_to_z, z_to_L0, conditional_ODE_func, decoder
from models import LUPI_NDP, NDP
models = [torch.load('../data/saved_models/'+model) for model in config['saved_models']]

In [81]:
if config['use_saved_models']:
    print(1)

1


In [80]:
from torch.utils.data import DataLoader
train_set = torch.load('../data/datasets/'+config['saved_train'])
train_loader = DataLoader(train_set, batch_size=1)

In [14]:
eval_set = OscillatorsGenerator(config, train=False)

Evaluation data generation...
Fixed paramters {'m1': 1.0, 'm2': 1.0, 'x1': 1.0, 'x2': -1.0, 'v1': 0, 'v2': 0, 'k': 0.5}
Varying parameters {'c': [0.5, 2.0]}


In [94]:
eval_loader = DataLoader(eval_set)

In [96]:
for _ in eval_loader:
    1

In [99]:
a,b,c = False,False,False

In [103]:
a = True

In [104]:
if not(a or b or c):
    print(1)

In [36]:
config = {
    # data stuff
    'train_examples' : 500,
    'train_val_split' : 0.8,
    'eval_examples' : 500,
    'simulator' : {
        'dt' : 0.01,
        'sampling_freq' : 20,
        'samples' : 50
    },
    'fixed' : {
        'a' : 2/3,
        'b' : 4/3,
        'c' : 1,
        'd' : 1,
    },
    'varying' : {
        'u1' : [0.2, 1.],
        'v1' : [0.1, 0.5]
    },
    'eval_varying' : {
        'u1' : [0.2, 1.],
        'v1' : [0.1, 0.5]
    },
}

In [37]:
train_val_set = LotkaVolterraGenerator(config)
train_set = train_val_set[:int(config['train_examples']*config['train_val_split'])]
val_set = train_val_set[int(config['train_examples']*config['train_val_split']):]
eval_set = LotkaVolterraGenerator(config, train=False)

Training data generation...
Fixed paramters {'a': 0.6666666666666666, 'b': 1.3333333333333333, 'c': 1, 'd': 1}
Varying parameters {'u1': [0.2, 1.0], 'v1': [0.1, 0.5]}
Evaluation data generation...
Fixed paramters {'a': 0.6666666666666666, 'b': 1.3333333333333333, 'c': 1, 'd': 1}
Varying parameters {'u1': [0.2, 1.0], 'v1': [0.1, 0.5]}


In [38]:
torch.save(train_set, 'lotka_volterra_train')
torch.save(val_set, 'lotka_volterra_validation')
torch.save(eval_set, 'lotka_volterra_evaluation')

In [26]:
vs = torch.load('vs_eval_set')

In [53]:
save = 'nopi-exp-2021-02-19-1402'
pickle.load(open('../notebooks/saves/'+save+'-config.p', 'rb'))

{'model': 'nopi',
 'train_examples': 500,
 'train_val_split': 0.8,
 'eval_examples': 100,
 'simulator': {'dt': 0.01, 'sampling_freq': 20, 'samples': 50},
 'fixed': {'a': 0.6666666666666666, 'b': 1.3333333333333333, 'c': 1, 'd': 1},
 'varying': {'amp': [-1, 1], 'shift': [-0.5, 0.5]},
 'eval_varying': {'amp': [1, 1.1], 'shift': [0.5, 0.6]},
 'x_dim': 1,
 'obs_enc_h_dim': 16,
 'r_dim': 16,
 'pi_dim': 2,
 'pi_enc_h_dim': 16,
 'pi_r_dim': 16,
 'aggregators': ['logsumexp', 'mean'],
 'agg_h_dim': 16,
 'r_to_z_h_dim': 16,
 'z_dim': 16,
 'z_to_L_h_dim': 16,
 'L_dim': 16,
 'ode_h_dim': 16,
 'dec_h_dim': 16,
 'lr': 0.001,
 'epochs': 100,
 'val_and_print': 10,
 'context_range': [5, 10],
 'target_range': [15, 51],
 'save_model': True}